In [ ]:
import tensorflow as tf
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

def image_preprocessing(image,size):
    image = tf.image.resize(image, (size, size))
    image = tf.cast(image, tf.float32)
    image = image / 255.0
    return image

In [1]:
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# Load pre-trained Xception model (without top layers)
base_model = Xception(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add new classification layers
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(1024, activation='relu')(x)  # Add additional dense layers if needed
predictions = Dense(num_classes, activation='softmax')(x)  # 'num_classes' is the number of classes in your dataset

# Create the model
model = tf.keras.Model(inputs=base_model.input, outputs=predictions)

# Freeze base model layers (optional)
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Fine-tune the model
model.fit(train_dataset,
          epochs=10,
          validation_data=val_dataset)


test
